In this notebook we are performing all the feature engineering steps. And the end of the notebook we are creating a csv file with all the new features in that file so that we don't need to run this file every time. 

In [2]:
# needed libraries 

#!pip install textblob
import os
import pandas as pd
import re
import numpy as np
import spacy
from collections import Counter
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords

In [26]:
# uncomment for the first time only  
# !python -m spacy download en
import en_core_web_sm
nlp = en_core_web_sm.load()

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.2/12.8 MB 3.3 MB/s eta 0:00:04
     - -------------------------------------- 0.4/12.8 MB 4.4 MB/s eta 0:00:03
     -- ------------------------------------- 0.7/12.8 MB 4.9 MB/s eta 0:00:03
     --- ------------------------------------ 1.0/12.8 MB 5.3 MB/s eta 0:00:03
     --- ------------------------------------ 1.2/12.8 MB 5.2 MB/s eta 0:00:03
     ---- ----------------------------------- 1.5/12.8 MB 5.3 MB/s eta 0:00:03
     ----- ---------------------------------- 1.8/12.8 MB 5.4 MB/s eta 0:00:03
     ------ --------------------------------- 2.1/12.8 MB 5.6 MB/s eta 0:00:02
     ------- -------------------------------- 2.4/12.8 MB 5.6 MB/s eta 0:00:02
     -------- ------------------------------- 2.7/12.8 MB 5.6 MB/s eta 0:00:02
     ---------


[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# uncomment for the first time only  
# import sys
# !{sys.executable} -m pip install spacy
# !{sys.executable} -m spacy download en

     ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
     ---------------------------------------- 0.1/12.2 MB 2.4 MB/s eta 0:00:06
      --------------------------------------- 0.3/12.2 MB 2.9 MB/s eta 0:00:05
     - -------------------------------------- 0.5/12.2 MB 3.8 MB/s eta 0:00:04
     -- ------------------------------------- 0.8/12.2 MB 4.4 MB/s eta 0:00:03
     --- ------------------------------------ 1.1/12.2 MB 4.7 MB/s eta 0:00:03
     ---- ----------------------------------- 1.4/12.2 MB 4.9 MB/s eta 0:00:03
     ----- ---------------------------------- 1.7/12.2 MB 5.2 MB/s eta 0:00:03
     ------ --------------------------------- 2.0/12.2 MB 5.5 MB/s eta 0:00:02
     ------- -------------------------------- 2.3/12.2 MB 5.5 MB/s eta 0:00:02
     -------- ------------------------------- 2.6/12.2 MB 5.6 MB/s eta 0:00:02
     --------- ------------------------------ 2.9/12.2 MB 5.7 MB/s eta 0:00:02
     ---------- ----------------------------- 3.1/12.2 MB 5


[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.1/12.8 MB 2.4 MB/s eta 0:00:06
     - -------------------------------------- 0.3/12.8 MB 3.4 MB/s eta 0:00:04
     - -------------------------------------- 0.6/12.8 MB 4.3 MB/s eta 0:00:03
     -- ------------------------------------- 0.9/12.8 MB 4.9 MB/s eta 0:00:03
     --- ------------------------------------ 1.2/12.8 MB 5.1 MB/s eta 0:00:03
     ---- ----------------------------------- 1.5/12.8 MB 5.5 MB/s eta 0:00:03
     ----- ---------------------------------- 1.8/12.8 MB 5.6 MB/s eta 0:00:02
     ------ --------------------------------- 2.1/12.8 MB 5.7 MB/s eta 0:00:02
     ------- -------------------------------- 2.4/12.8 MB 5.7 MB/s eta 0:00:02
     -------- ------------------------------- 2.7/12.8 MB 5.7 MB/s eta 0:00:02
     ---------


[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# needed libraries

import nltk
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\neels\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\neels\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\neels\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\neels\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [12]:
# reading the csv file which we created in Step 2

df = pd.read_csv("enron.csv")
df = df.drop(["Email Folder"], axis=1)
df

,Unnamed: 0,Author,Folder,File,Message ID,Text,Raw Text
0,0,Phillip K,allen-p,1,18782981.1075855378110.,Here is our forecast\n\n,Message-ID: <18782981.1075855378110.JavaMail.e...
1,1,Phillip K,allen-p,10,15464986.1075855378456.,Traveling to have a business meeting takes the...,Message-ID: <15464986.1075855378456.JavaMail.e...
2,2,Phillip K,allen-p,100,24216240.1075855687451.,test successful. way to go!!!,Message-ID: <24216240.1075855687451.JavaMail.e...
3,3,Phillip K,allen-p,1000,13505866.1075863688222.,"Randy,\n\n Can you send me a schedule of the s...",Message-ID: <13505866.1075863688222.JavaMail.e...
4,4,Phillip K,allen-p,1001,30922949.1075863688243.,Let's shoot for Tuesday at 11:45.,Message-ID: <30922949.1075863688243.JavaMail.e...
...,...,...,...,...,...,...,...
113346,333,"Zufferli, John",zufferli-j,95,26807948.1075842029936.,This is a trade with OIL-SPEC-HEDGE-NG (John L...,Message-ID: <26807948.1075842029936.JavaMail.e...
113347,334,"Zufferli, John",zufferli-j,96,25835861.1075842029959.,Some of my position is with the Alberta Term b...,Message-ID: <25835861.1075842029959.JavaMail.e...
113348,335,"Zufferli, John",zufferli-j,97,28979867.1075842029988.,2\n\n,Message-ID: <28979867.1075842029988.JavaMail.e...
113349,336,"Zufferli, John",zufferli-j,98,22052556.1075842030013.,Analyst\t\t\t\t\tRank\n\nStephane Brodeur\t\t\...,Message-ID: <22052556.1075842030013.JavaMail.e...


In [13]:
# here we are droping uncertain data

top_authors = df.value_counts(["Folder"])[:5]
df = df.loc[df["Folder"].isin(list(top_authors.index.get_level_values(0)))].drop(["Unnamed: 0"], axis=1).reset_index(drop=True)
df = df[df["Text"]!=" "]
df = df[df["Text"]!="\n"]
df = df.dropna()
df

,Author,Folder,File,Message ID,Text,Raw Text
0,Jeff,dasovich-j,1,9042044.1075843184295.,Will we be covering this? The utilities in Ca...,Message-ID: <9042044.1075843184295.JavaMail.ev...
1,Jeff,dasovich-j,10,24845688.1075843184521.,When's a good time to discuss? I'd like to ge...,Message-ID: <24845688.1075843184521.JavaMail.e...
2,Jeff,dasovich-j,100,10392081.1075843186589.,what do you mean high?\n\n,Message-ID: <10392081.1075843186589.JavaMail.e...
3,Jeff,dasovich-j,1000,29295435.1075843207818.,FYI.\n,Message-ID: <29295435.1075843207818.JavaMail.e...
4,Jeff,dasovich-j,1001,24205254.1075843207840.,FYI.\n,Message-ID: <24205254.1075843207840.JavaMail.e...
...,...,...,...,...,...,...
28102,"Shackleton, Sara",shackleton-s,95,4063601.1075858807900.,Kim: Sorry for the mixup. Attached is my red...,Message-ID: <4063601.1075858807900.JavaMail.ev...
28103,"Shackleton, Sara",shackleton-s,96,13680534.1075858807922.,Attached is the list which I created when Mark...,Message-ID: <13680534.1075858807922.JavaMail.e...
28104,"Shackleton, Sara",shackleton-s,97,13774859.1075858807944.,Vijay:\n\nI've been trying to call you for abo...,Message-ID: <13774859.1075858807944.JavaMail.e...
28105,"Shackleton, Sara",shackleton-s,98,22034730.1075858807967.,Tanya: FTU has asked to resume negotiations o...,Message-ID: <22034730.1075858807967.JavaMail.e...


In [15]:
# this function is used to make sure that our dataset is proportinal and not skewed 

def uniform_distribution(samples_per_author, df):
    df3 = pd.DataFrame(columns=["Author", "Folder", "File", "Text", "Raw Text"]) 
    for folder in df["Folder"].value_counts().index:
        df3 = df3.append(df[df["Folder"]==folder].sample(n=samples_per_author), ignore_index=True)
    return df3

In [16]:
df = uniform_distribution(4000, df)
print(df["Folder"].value_counts())
df

mann-k          4000
kaminski-v      4000
dasovich-j      4000
germany-c       4000
shackleton-s    4000
Name: Folder, dtype: int64


C:\Users\neels\AppData\Local\Temp\ipykernel_6800\2072962815.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df3.append(df[df["Folder"]==folder].sample(n=samples_per_author), ignore_index=True)
C:\Users\neels\AppData\Local\Temp\ipykernel_6800\2072962815.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df3.append(df[df["Folder"]==folder].sample(n=samples_per_author), ignore_index=True)
C:\Users\neels\AppData\Local\Temp\ipykernel_6800\2072962815.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df3.append(df[df["Folder"]==folder].sample(n=samples_per_author), ignore_index=True)
C:\Users\neels\AppData\Local\Temp\ipykernel_6800\2072962815.py:4: FutureWarning: The frame.append method is deprecated and w

,Author,Folder,File,Text,Raw Text,Message ID
0,Kay,mann-k,2120,Sounds good. Mike will be here in a bit to si...,Message-ID: <5580747.1075846079743.JavaMail.ev...,5580747.1075846079743.
1,Kay,mann-k,1086,"Chris Calger wants the financing CP removed, a...",Message-ID: <13545145.1075846054680.JavaMail.e...,13545145.1075846054680.
2,Kay,mann-k,462,I may be on the road tomorrow (won't know unt...,Message-ID: <22463406.1075845929642.JavaMail.e...,22463406.1075845929642.
3,Kay,mann-k,4158,"Ben/Mike,\n\nDid you guys work out anything on...",Message-ID: <29896220.1075846130416.JavaMail.e...,29896220.1075846130416.
4,Kay,mann-k,3764,Here it is:\n\n,Message-ID: <10715976.1075846120550.JavaMail.e...,10715976.1075846120550.
...,...,...,...,...,...,...
19995,Sara,shackleton-s,5802,You need to speak with the developer. There i...,Message-ID: <5377992.1075844916317.JavaMail.ev...,5377992.1075844916317.
19996,"Shackleton, Sara",shackleton-s,473,"Mark:\n\nGreat! Tuesday, March 19, at 2 pm, l...",Message-ID: <28808320.1075861053027.JavaMail.e...,28808320.1075861053027.
19997,"Shackleton, Sara",shackleton-s,715,Joanne: I don't believe I ever got this file....,Message-ID: <19706373.1075863196724.JavaMail.e...,19706373.1075863196724.
19998,Sara,shackleton-s,5542,Sheila: This agreement is for what futures pr...,Message-ID: <7557414.1075844909837.JavaMail.ev...,7557414.1075844909837.


In [17]:
#function to create a new attribute - no of characters in email (blank space are not counted) 
def feature_email_length_characters(text):
    if type(text) == str:
        text = text.strip()
        text = re.sub(r"\W", "", text)
        length = len(text)
        return length
    else:
        return np.NaN

#function to create a new attribute - ratio of digits to total length 
def feature_digit_density(text):
    if type(text) == str:
        text = text.strip()
        text = re.sub(r"\W", "", text)
        total_length = len(text)
        number_digits = len(re.findall(r"\d", text))
        try:
            return (number_digits/total_length)
        except ZeroDivisionError as e:
            return 0
    if type(text) == float:
        return np.NaN

#function to create a new attribute - ratio of blank space to total length
def feature_space_density(text):
    if type(text) == str:
        number_space = len(re.findall(r"[\s\n\t]", text))
        text = text.strip()
        text = re.sub(r"\W", "", text)
        total_length = len(text)
        try:
            return (number_space/total_length)
        except ZeroDivisionError as e:
            return 0
    if type(text) == float:
        return np.NaN

#function to create a new attribute - no of paras in email
def feature_paragraph(text):
    if type(text) == str:
        number_paragraphs = len(re.findall(r"\n\n", text))
        return max(1.0, number_paragraphs)
    if type(text) == float:
        return np.NaN
    
#function to create a new attribute - no of sentences in email
def feature_average_characters_paragraph(text):
    if type(text) == str:
        number_paragraphs = len(re.findall(r"\n\n", text))
        if not number_paragraphs:
            return max(1, len(re.findall(r"[.?!]\W", text)))
        else:
            paragraphs = re.findall(r"(?:.+\n)+\n", text)
            length = 0
            for paragraph in paragraphs:
                length += max(1, len(re.findall(r"[.?!]\W", paragraph)))
            return length/number_paragraphs
    if type(text) == float:
        return np.NaN
    

#function to create a new attribute - greetings in email
def feature_farewell_words(text):
    if type(text) == str:
        try:
            words = text.split()
            for word in reversed(words):
                if re.search(r"\w+", word):
                    last_word = re.search(r"\w+", word).group().lower()
                    return last_word
                else:
                    continue
            return "\n"
        except IndexError as e:
            return np.NaN
    if type(text) == float:
        return np.NaN

def feature_freq_farewell_words(farewell_words, text):
    if type(text) == str:
        if text in farewell_words:
            return text
        else:
            return "Other"
    if type(text) == float:
        return np.NaN
        

#function to create a new attribute - last punctuation mark in email
def feature_ending_punctuation(text):
    if type(text) == str:
        if re.search(r"[\!\,\.\?\:\'\"]", text):
            try:
                last_punc = re.findall(r"[\!\,\.\?\:\'\"]", text)[-1]
                return last_punc
            except IndexError as e:
                last_punc = len(re.findall(r"[\!\,\.\?\:\'\"]", text))
                if len == 1:
                    return last_punc[0]
        else:
            return "None"
    if type(text) == float:
        return np.NaN

##function to create a new attribute - most used punctuation mark in email
def feature_most_used_punctuation(text):
    if type(text) == str:
        if re.search(r"[\!\,\.\?\:\'\"]", text):
            punc = re.findall(r"[\!\,\.\?\:\'\"]", text)
            most_used_punc = Counter(punc).most_common(1)[0][0]
            return most_used_punc
        else:
            return "None"
    if type(text) == float:
        return np.NaN
    
#function to create a new attribute - using TextBlob calculating subjectivity
def feature_subjectivity(text):
    if type(text) == str:
        blob = TextBlob(text)
        return blob.sentiment.subjectivity
    if type(text) == float:
        return np.NaN

#function to create a new attribute - using TextBlob calculating polarity & subjectivity
def feature_polarity(text):
    if type(text) == str:
        blob = TextBlob(text)
        return blob.sentiment.polarity
    if type(text) == float:
        return np.NaN

#function to create a new attribute - most used POS in email
def feature_most_pos(text):
    if type(text) == str:
        blob = TextBlob(text)
        final_pos = []
        for word, pos in blob.tags:
            if word not in stopwords.words("english"):
                final_pos.append(pos)
        count_pos = Counter(final_pos)
        if count_pos.most_common():
            return count_pos.most_common()[0][0]
        else:
            return "Other"
    if type(text) == float:
        return np.NaN

##function to create a new attribute - get greeting words in email
def feature_get_greeting(text):
    if type(text) == str:
        if re.match(r"^\w+", text):
            greeting_word = re.match(r"^\w+", text).group()
            return greeting_word
        else:
            return "None"
    if type(text) == float:
        return np.NaN

##function to create a new attribute - finding the most commom word in email
def feature_most_common_word(text):
    if type(text) == str:
        blob = TextBlob(text)
        words = []
        for word, pos in blob.tags:
            words.append(word)
        count_word = Counter(words)
        if count_word.most_common():
            return count_word.most_common()[0][0]
        else:
            return np.NaN
    if type(text) == float:
        return np.NaN

def feature_freq_most_common_word(text):
    if type(text) == str:
        blob = TextBlob(text)
        words = []
        for word, pos in blob.tags:
            words.append(word)
        count_word = Counter(words)
        if count_word.most_common():
            return count_word.most_common()[0][1]
        else:
            return 0
    if type(text) == float:
        return np.NaN

def feature_number_words(text):
    if type(text) == str:
        blob = TextBlob(text)
        return len(blob.words)
    if type(text) == float:
        return np.NaN


In [18]:
#function to create a new attribute - avg length of word in email
def avg_length(text):
    if type(text) == str:
        list1 = text.split()
        word_len = 0
        for text in list1:
            strip_text=text.strip()
            if re.search(r"\w+",strip_text):
                strip_text = re.search(r"\w+",strip_text).group()
                word_len += len(strip_text)
        try:
            avg_word_len = word_len/len(list1)
        except ZeroDivisionError as e:
            return np.NaN
        return avg_word_len
    else:
        return np.NaN

#function to create a new attribute - avg length of sentence in email
def avg_sentence_length(text):
    if type(text) == str:
        list1 = re.findall(r"[^\.\?\!]+",text)
        sent_len = 0 
        for le in list1:
            sent_len += len(le)
        try:
            return sent_len/len(list1) 
        except:
            return np.NaN      
    else:
        return np.NaN

#function to create a new attribute - ratio of short words to long words in email
def feature_short_word_ratio(text):
    if type(text) == str:
        list1 = text.split()
        short_word = 0
        if len(list1) >= 1:
            for word in list1:
                strip_text = word.strip()
                if re.search(r"\w+",strip_text):
                    strip_text = re.search(r"\w+",strip_text).group()
                    word_len = len(strip_text)
                    if word_len < 4:
                        short_word +=1
            try:
                short_word_rat = short_word/len(list1)
                return short_word_rat
            except:
                return 0
        else:
            return 0  
    else:
        return np.NaN

#function to create a new attribute - count of punctuation in email
def punctuation_frequency(text):
    if type(text) == str:
        if re.search(r"[\!\,\.\?\:\'\"]", text):
            punc = re.findall(r"[\!\,\.\?\:\'\"]", text)
            freq_punc = len(punc)
            return freq_punc
        else:
            return 0
    if type(text) == float:
        return np.NaN

##function to create a new attribute - extrating punctuation after greetings in email
def punctuation_greeting(text):
    if type(text) == str:
        if re.search(r"^\w+([\,\:\?\!\-])\n", text):
            punc = re.search(r"^\w+([\,\:\?\!\-])\n", text).group(1)
            return punc
        else:
            return "None"
    if type(text) == float:
        return np.NaN

#function to create a new attribute - special characters in email
def feature_number_special_characters(text):
    if type(text) == str:
        special_characters = re.findall(r"[\@\#\$\%\^\&\~\`\*\(\)\<\>\\\[\]\{\}\|]", text)
        return len(special_characters)
    if type(text) == float:
        return np.NaN

#function to create a new attribute - maximum count of special characters in email
def feature_max_special_character(text):
    if type(text) == str:
        special_characters = re.findall(r"[\@\#\$\%\^\&\~\`\*\(\)\<\>\\\[\]\{\}\|]", text)
        special_char_count = Counter(special_characters)
        if special_char_count.most_common():
            max_special_char = special_char_count.most_common()[0][0]
            return max_special_char
        else:
            return "None"
    if type(text) == float:
        return np.NaN

def feature_freq_max_special_character(text):
    if type(text) == str:
        special_characters = re.findall(r"[\@\#\$\%\^\&\~\`\*\(\)\<\>\\\[\]\{\}\|]", text)
        special_char_count = Counter(special_characters)
        if special_char_count.most_common():
            freq_max_special_char = special_char_count.most_common()[0][1]
            return freq_max_special_char
        else:
            return "0"
    if type(text) == float:
        return np.NaN


In [19]:
# for each sentence in mail we check the max no
  
def check_single_sentence(clean_text):
    if type(clean_text) == str:
        ending_punc = re.findall(r"[.?!]", clean_text)
        ending_punc_count = Counter(ending_punc)
        single_sentence = False
        if(ending_punc_count):
            max_ep_char = max(ending_punc_count, key=ending_punc_count.get)
            max_ep_value = max(ending_punc_count.values())
        else:
            max_ep_char = ''
            max_ep_value = 0
        if max_ep_value<=1:
            single_sentence = True
    elif type(clean_text) == float:
        return np.NaN 
    return single_sentence



In [20]:
# perfoming feature engineering for no of characters in email
email_length = df["Text"].apply(lambda row: feature_email_length_characters(row))
email_length.dropna()

0         45
1        100
2         94
3         51
4          8
        ... 
19995    338
19996     48
19997     44
19998    217
19999    246
Name: Text, Length: 20000, dtype: int64

In [21]:
# perfoming feature engineering for ratio of digits to characters in email
digit_density = df["Text"].apply(lambda row: feature_digit_density(row))
digit_density.dropna()

0        0.000000
1        0.030000
2        0.000000
3        0.039216
4        0.000000
           ...   
19995    0.097633
19996    0.062500
19997    0.000000
19998    0.152074
19999    0.134146
Name: Text, Length: 20000, dtype: float64

In [22]:
# perfoming feature engineering for ratio of white spaces to characters in email

space_density = df["Text"].apply(lambda row: feature_space_density(row))
space_density.dropna()

0        0.377778
1        0.270000
2        0.276596
3        0.294118
4        0.500000
           ...   
19995    0.227811
19996    0.354167
19997    0.363636
19998    0.221198
19999    0.235772
Name: Text, Length: 20000, dtype: float64

In [23]:
# perfoming feature engineering for no of paras in email
number_paragraphs = df["Text"].apply(lambda row: feature_paragraph(row))
number_paragraphs.dropna()

0        3.0
1        1.0
2        2.0
3        3.0
4        1.0
        ... 
19995    3.0
19996    2.0
19997    1.0
19998    1.0
19999    2.0
Name: Text, Length: 20000, dtype: float64

In [24]:
# perfoming feature engineering for no of characters per para in email
average_sentences_paragraph = df["Text"].apply(lambda row: feature_average_characters_paragraph(row))
average_sentences_paragraph.dropna()

0        1.333333
1        1.000000
2        1.000000
3        1.000000
4        1.000000
           ...   
19995    1.666667
19996    1.500000
19997    2.000000
19998    3.000000
19999    2.500000
Name: Text, Length: 20000, dtype: float64

In [27]:
farewell_words = df["Text"].apply(lambda row: feature_farewell_words(row))
raw_freq_farewell_words = farewell_words.value_counts()
print(raw_freq_farewell_words)
raw_freq_farewell_words = list(raw_freq_farewell_words[raw_freq_farewell_words>20].index)
print(raw_freq_farewell_words)
freq_farewell_words = []
for word in raw_freq_farewell_words:
    tokens = nlp(word)
    for token in tokens:
        if token.pos_ not in ["PROPN"]:
            freq_farewell_words.append(token.text)

farewell_words = farewell_words.apply(lambda row: feature_freq_farewell_words(freq_farewell_words, row))
farewell_words.dropna()

vince       2676
sara        1566
kay         1479
jeff        1192
thanks       752
            ... 
socalgas       1
relief         1
wow            1
oops           1
shari          1
Name: Text, Length: 2834, dtype: int64
['vince', 'sara', 'kay', 'jeff', 'thanks', 'best', 'ss', 'fyi', '853', 'you', 'it', 'http', 'fax', 'me', 'kaminski', 'ckm', 'this', 'the', 'to', 'now', 'questions', 'that', 'today', 'know', '\n', 'time', 'tomorrow', '713', 'week', 'day', 'please', 'think', 'on', 'is', 'too', 'call', 'yet', 'a', 'out', 'there', 'meeting', 'deal', 'do', 'comments', 'much', 'for', '4', 'ok', '3', 'yes', 'i', 'of', 'one', 'well', 'help', 'deserved', '646', '415', 'up', 'good', 'in', '2', 'again', 'and', 'am', 'soon', 'be', '1', 'here', 'done', 'agreement', 'though', 'man', 'email', 'them', 'him', 'with', 'monday', 'morning', 'weekend', 'number', 'friday', 'contract', 'later', 'right', 'stuff', 'work', 'list', '20', 'go']


0         Other
1         Other
2        thanks
3        thanks
4            is
          ...  
19995     Other
19996     Other
19997     Other
19998       fax
19999       fax
Name: Text, Length: 20000, dtype: object

In [28]:
# perfoming feature engineering for extracting greeting words in email
greeting_words = df["Text"].apply(lambda row: feature_get_greeting(row))
greeting_words.dropna()

0        Sounds
1         Chris
2             I
3           Ben
4          Here
          ...  
19995       You
19996      Mark
19997    Joanne
19998    Sheila
19999    Cheryl
Name: Text, Length: 20000, dtype: object

In [29]:
# perfoming feature engineering for most common word in email
most_common_word = df["Text"].apply(lambda row: feature_most_common_word(row))
most_common_word.dropna()

0          Sounds
1              we
2              be
3        Ben/Mike
4            Here
           ...   
19995         the
19996        Mark
19997           I
19998        Sara
19999       email
Name: Text, Length: 19943, dtype: object

In [30]:
most_common_word[most_common_word.isna()==True]
df.iloc[1714]

Author                                                      Kay
Folder                                                   mann-k
File                                                       2786
Text                     Sounds like a good approach to me.\n\n
Raw Text      Message-ID: <30919464.1075845986197.JavaMail.e...
Message ID                              30919464.1075845986197.
Name: 1714, dtype: object

In [31]:
# finding polarity and subjectivity of each sentences 

subjectivity = df["Text"].apply(lambda row: feature_subjectivity(row))
polarity = df["Text"].apply(lambda row: feature_polarity(row))
print(subjectivity.dropna())
print(polarity.dropna())

0        0.400000
1        0.100000
2        0.200000
3        0.200000
4        0.000000
           ...   
19995    0.363333
19996    0.750000
19997    0.200000
19998    0.200000
19999    0.100000
Name: Text, Length: 20000, dtype: float64
0        0.450000
1        0.200000
2        0.325000
3        0.200000
4        0.000000
           ...   
19995   -0.183333
19996    1.000000
19997    0.200000
19998    0.200000
19999    0.100000
Name: Text, Length: 20000, dtype: float64


In [32]:
# perfoming feature engineering for no of most coomon word in email

freq_most_common_word = df["Text"].apply(lambda row: feature_freq_most_common_word(row))
freq_most_common_word.dropna()

0        1
1        2
2        2
3        1
4        1
        ..
19995    5
19996    1
19997    2
19998    2
19999    3
Name: Text, Length: 20000, dtype: int64

In [33]:
# perfoming feature engineering for most of POS in email

pos = df["Text"].apply(lambda row: feature_most_pos(row))
pos


0        NNS
1        NNP
2         NN
3        NNP
4         RB
        ... 
19995     NN
19996     NN
19997     NN
19998    NNP
19999    NNP
Name: Text, Length: 20000, dtype: object

In [34]:
pos.dropna()

0        NNS
1        NNP
2         NN
3        NNP
4         RB
        ... 
19995     NN
19996     NN
19997     NN
19998    NNP
19999    NNP
Name: Text, Length: 20000, dtype: object

In [35]:
# perfoming feature engineering for exracting the last punctuation mark in email

last_punc = df["Text"].apply(lambda row: feature_ending_punctuation(row))
last_punc.dropna()

0        ,
1        .
2        ,
3        ,
4        :
        ..
19995    .
19996    .
19997    .
19998    ,
19999    ,
Name: Text, Length: 20000, dtype: object

In [36]:
# perfoming feature engineering for no of punctuation marks in email

freq_punc = df["Text"].apply(lambda row: feature_most_used_punctuation(row))
freq_punc.dropna()


0        .
1        ,
2        ,
3        ,
4        :
        ..
19995    .
19996    ,
19997    .
19998    ,
19999    .
Name: Text, Length: 20000, dtype: object

In [37]:
# finding the avg length 

avg_len = df["Text"].apply(lambda row: avg_length(row))
avg_len


0        3.461538
1        4.000000
2        4.227273
3        3.916667
4        2.666667
           ...   
19995    4.500000
19996    3.692308
19997    3.909091
19998    4.853659
19999    4.509804
Name: Text, Length: 20000, dtype: float64

In [38]:
# finding the avg sentence length 

avg_sent_len = df["Text"].apply(lambda row: avg_sentence_length(row))
avg_sent_len

0        21.000000
1        64.000000
2        62.500000
3        34.500000
4        13.000000
           ...    
19995    53.875000
19996    23.000000
19997    20.666667
19998    55.600000
19999    53.166667
Name: Text, Length: 20000, dtype: float64

In [39]:
# finding the ratio of short words to long words 

short_word_ratio = df["Text"].apply(lambda row: feature_short_word_ratio(row))
short_word_ratio.dropna()

0        0.461538
1        0.560000
2        0.454545
3        0.500000
4        0.666667
           ...   
19995    0.455882
19996    0.538462
19997    0.363636
19998    0.268293
19999    0.352941
Name: Text, Length: 20000, dtype: float64

In [40]:
# finding the no of punctuation marks

punc_freq = df["Text"].apply(lambda row: punctuation_frequency(row))
punc_freq

0         3
1         2
2         4
3         3
4         1
         ..
19995    13
19996     6
19997     4
19998     8
19999    12
Name: Text, Length: 20000, dtype: int64

In [41]:
# finding the punctuation mark after greeting 

punc_greet = df["Text"].apply(lambda row: punctuation_greeting(row))
punc_greet.dropna()

0        None
1        None
2        None
3        None
4        None
         ... 
19995    None
19996       :
19997    None
19998    None
19999       :
Name: Text, Length: 20000, dtype: object

In [42]:
number_words = df["Text"].apply(lambda row: feature_number_words(row))
number_words

0        13
1        25
2        23
3        12
4         3
         ..
19995    69
19996    13
19997    12
19998    41
19999    52
Name: Text, Length: 20000, dtype: int64

In [43]:
number_special_characters = df["Text"].apply(lambda row: feature_number_special_characters(row))
print(number_special_characters.dropna())
max_special_character = df["Text"].apply(lambda row: feature_max_special_character(row))
print(max_special_character.dropna())
freq_max_special_character = df["Text"].apply(lambda row: feature_freq_max_special_character(row))
print(freq_max_special_character.dropna())

single_sentence = df["Text"].apply(lambda row: check_single_sentence(row))
print(single_sentence.dropna())

0        0
1        0
2        2
3        0
4        0
        ..
19995    5
19996    0
19997    0
19998    4
19999    4
Name: Text, Length: 20000, dtype: int64
0        None
1        None
2           (
3        None
4        None
         ... 
19995       (
19996    None
19997    None
19998       (
19999       (
Name: Text, Length: 20000, dtype: object
0        0
1        0
2        1
3        0
4        0
        ..
19995    2
19996    0
19997    0
19998    2
19999    2
Name: Text, Length: 20000, dtype: object
0        False
1         True
2         True
3         True
4         True
         ...  
19995    False
19996     True
19997    False
19998    False
19999    False
Name: Text, Length: 20000, dtype: bool


In [44]:
df.loc[0, ["Text", "File", "Folder"]]

Text      Sounds good.  Mike will be here in a bit to si...
File                                                   2120
Folder                                               mann-k
Name: 0, dtype: object

In [45]:
# combining all the features we thus far created 

df["Email Length"] = email_length
df["Digit Density"] = digit_density
df["Space Density"] = space_density
df["Number of Paragraphs"] = number_paragraphs
df["Average Sentences per Paragraph"] = average_sentences_paragraph
df["Farewell Words"] = farewell_words
df["Freq Punc"] = freq_punc
df["Last Punc"] = last_punc
df["Average Word Length"] = avg_len
df["Average Sentence Length"] = avg_sent_len
df["Short Word Ratio"] = short_word_ratio
df["Punc Frequency"] = punc_freq
df["Punc after Greeting"] = punc_greet
df["Number Words"] = number_words
df["Subjectivity"] = subjectivity
df["Polarity"] = polarity
df["Most Common POS"] = pos
df["Single Sentence"] = single_sentence
df["Greeting"] = greeting_words
df["Most Common Word"] = most_common_word
df["Freq Most Common Word"] = freq_most_common_word
df["Total Special Character Count"] = number_special_characters
df["Max Occurring Special Char"] = max_special_character
df["Count of Max Special Char"] = freq_max_special_character

In [46]:
df

,Author,Folder,File,Text,Raw Text,Message ID,Email Length,Digit Density,Space Density,Number of Paragraphs,...,Subjectivity,Polarity,Most Common POS,Single Sentence,Greeting,Most Common Word,Freq Most Common Word,Total Special Character Count,Max Occurring Special Char,Count of Max Special Char
0,Kay,mann-k,2120,Sounds good. Mike will be here in a bit to si...,Message-ID: <5580747.1075846079743.JavaMail.ev...,5580747.1075846079743.,45,0.000000,0.377778,3.0,...,0.400000,0.450000,NNS,False,Sounds,Sounds,1,0,None,0
1,Kay,mann-k,1086,"Chris Calger wants the financing CP removed, a...",Message-ID: <13545145.1075846054680.JavaMail.e...,13545145.1075846054680.,100,0.030000,0.270000,1.0,...,0.100000,0.200000,NNP,True,Chris,we,2,0,None,0
2,Kay,mann-k,462,I may be on the road tomorrow (won't know unt...,Message-ID: <22463406.1075845929642.JavaMail.e...,22463406.1075845929642.,94,0.000000,0.276596,2.0,...,0.200000,0.325000,NN,True,I,be,2,2,(,1
3,Kay,mann-k,4158,"Ben/Mike,\n\nDid you guys work out anything on...",Message-ID: <29896220.1075846130416.JavaMail.e...,29896220.1075846130416.,51,0.039216,0.294118,3.0,...,0.200000,0.200000,NNP,True,Ben,Ben/Mike,1,0,None,0
4,Kay,mann-k,3764,Here it is:\n\n,Message-ID: <10715976.1075846120550.JavaMail.e...,10715976.1075846120550.,8,0.000000,0.500000,1.0,...,0.000000,0.000000,RB,True,Here,Here,1,0,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,Sara,shackleton-s,5802,You need to speak with the developer. There i...,Message-ID: <5377992.1075844916317.JavaMail.ev...,5377992.1075844916317.,338,0.097633,0.227811,3.0,...,0.363333,-0.183333,NN,False,You,the,5,5,(,2
19996,"Shackleton, Sara",shackleton-s,473,"Mark:\n\nGreat! Tuesday, March 19, at 2 pm, l...",Message-ID: <28808320.1075861053027.JavaMail.e...,28808320.1075861053027.,48,0.062500,0.354167,2.0,...,0.750000,1.000000,NN,True,Mark,Mark,1,0,None,0
19997,"Shackleton, Sara",shackleton-s,715,Joanne: I don't believe I ever got this file....,Message-ID: <19706373.1075863196724.JavaMail.e...,19706373.1075863196724.,44,0.000000,0.363636,1.0,...,0.200000,0.200000,NN,False,Joanne,I,2,0,None,0
19998,Sara,shackleton-s,5542,Sheila: This agreement is for what futures pr...,Message-ID: <7557414.1075844909837.JavaMail.ev...,7557414.1075844909837.,217,0.152074,0.221198,1.0,...,0.200000,0.200000,NNP,False,Sheila,Sara,2,4,(,2


In [47]:
# converting dataframe to csv so that we don't need to run these steps every time 

df.to_csv("Enron_29_Features.csv")
